<a href="https://colab.research.google.com/github/SahilCarterr/FinetuningLLM/blob/main/bart_large_mnli_Finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/data_withoutLENE.csv')

In [3]:
df.head()

,question,topic
0,Once your ex was your lover will you give them...,Dating and Relationships
1,Any,Random
2,What type of guys do u hate on hunch,Dating and Relationships
3,Karnapishachini ko kya chahiye,Movies and TV
4,Feeling hai woExpress nhi kar,Dating and Relationships


In [4]:
unique_topics = df['topic'].unique()
topic_mapping = {topic: i for i, topic in enumerate(unique_topics)}

# Apply the mapping to the 'topic' column
df['topic'] = df['topic'].map(topic_mapping)

# Display the updated DataFrame and the mapping of unique strings to numbers
print("Mapping of unique strings to numbers:", topic_mapping)

Mapping of unique strings to numbers: {'Dating and Relationships': 0, 'Random': 1, 'Movies and TV': 2, 'Music': 3, 'Technology and Internet': 4, 'Philosophy and Religion': 5, 'technology and Internet': 6, 'Law and Order': 7, 'Health and Wellness': 8, 'Food and Drinks': 9, 'Fashion and Makeup': 10, 'health and Wellness': 11, 'Gaming': 12, 'Sports': 13, 'fashion and Makeup': 14, 'sports': 15, 'Anime': 16, 'anime': 17, 'Finance': 18, 'Travel and Geography': 19, 'Pets and Animals': 20, 'gaming': 21, 'random': 22, 'music': 23}


In [5]:
df.head()

,question,topic
0,Once your ex was your lover will you give them...,0
1,Any,1
2,What type of guys do u hate on hunch,0
3,Karnapishachini ko kya chahiye,2
4,Feeling hai woExpress nhi kar,0


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Save the train and test datasets to new CSV files
train_data.to_csv('train.csv', index=False)
test_data.to_csv('test.csv', index=False)

print("Data has been split and saved to train.csv and test.csv.")


Data has been split and saved to train.csv and test.csv.


In [7]:
!pip -q install pyarrow datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [1]:
from datasets import load_dataset
data_files = {"train": "/content/train.csv", "test": "/content/test.csv"}
dataset = load_dataset("csv", data_files=data_files)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'topic'],
        num_rows: 1274
    })
    test: Dataset({
        features: ['question', 'topic'],
        num_rows: 319
    })
})

In [2]:
from transformers import AutoTokenizer, DataCollatorWithPadding
checkpoint = "facebook/bart-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    # Ensure inputs are strings, handle potential None or other non-string types
    questions = [str(q) if q is not None else "" for q in examples['question']]
    return tokenizer(questions,truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/1274 [00:00<?, ? examples/s]

Map:   0%|          | 0/319 [00:00<?, ? examples/s]

In [3]:
tokenized_datasets = tokenized_datasets.remove_columns(["question"])
tokenized_datasets = tokenized_datasets.rename_column("topic", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'attention_mask']

In [4]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)

In [5]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 35]),
 'attention_mask': torch.Size([8, 35])}

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=24,ignore_mismatched_sizes=True)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large-mnli and are newly initialized because the shapes did not match:
- classification_head.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([24]) in the model instantiated
- classification_head.out_proj.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([24, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(3.2433, grad_fn=<NllLossBackward0>) torch.Size([8, 24])


In [9]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

480


In [12]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [13]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/480 [00:00<?, ?it/s]

In [ ]:
!pip -q install evaluate

In [14]:
import torch

model.eval()
all_predictions = []
all_labels = []

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    # Append predictions and labels
    all_predictions.extend(predictions.cpu().numpy())
    all_labels.extend(batch['labels'].cpu().numpy())

# Convert lists to tensors
all_predictions = torch.tensor(all_predictions)
all_labels = torch.tensor(all_labels)

# Compute accuracy
accuracy = (all_predictions == all_labels).float().mean().item()

print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.7147
